In [1]:
import pandas as pd
train=pd.read_csv('./train_set.csv',sep='\t')
test=pd.read_csv('./test_a.csv',sep='\t')

In [2]:
#链接train和test
df_all = pd.concat([train,test])

In [4]:
!pip install scikit-learn

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,3),min_df=0.1,max_df=0.8)
#对text文本进行TIFID特征提取
text_features=vec.fit_transform(df_all['text'])
text_features

<250000x1262 sparse matrix of type '<class 'numpy.float64'>'
	with 72631705 stored elements in Compressed Sparse Row format>

In [4]:
df=pd.DataFrame(text_features.toarray(),columns=vec.get_feature_names())
df

,101,1018,1031,1031 761,1036,1043,1066,1070,1070 1036,1070 2107,...,913,920,922,922 885,94,955,965,974,979,980
0,0.0,0.000000,0.016014,0.019002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.023082,0.000000,0.000000,0.000000,0.000000,0.000000,0.017817,0.000000,0.000000,0.044522
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.038494,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.025567,0.000000,0.000000,0.000000,0.000000,0.043697,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.020046,0.000000,0.012311,0.109544,0.027356,0.030878,...,0.044219,0.000000,0.022074,0.000000,0.000000,0.021707,0.011378,0.025657,0.030931,0.000000
4,0.0,0.000000,0.000000,0.000000,0.000000,0.107275,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0.0,0.134798,0.000000,0.000000,0.000000,0.000000,0.092056,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.040581,0.000000,0.000000,0.000000,0.000000
249996,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010965,0.000000,0.000000,0.000000,0.000000,0.000000,0.016928,0.000000,0.000000,0.000000
249997,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021514,0.000000,0.322201,0.088809,0.034471,0.031685,0.000000,0.037449,0.000000,0.000000
249998,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087323,0.000000


In [5]:
df_train=df[df.index < len(train)]
df_test=df[df.index >= len(train)]

In [16]:
import pickle
df.to_pickle('./df.pkl')

In [ ]:
!pip install xgboost

In [43]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(
            max_depth=12, learning_rate=0.05, n_estimators=2000, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, random_state=2021,
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )

In [23]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Columns: 1262 entries, 101 to 980
dtypes: float64(1262)
memory usage: 1.9 GB


In [ ]:
model_xgb.fit(df_train,train['label'])

[21:21:37] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
result_xgb = model_xgb.predict_proba(df_test)

In [ ]:
import numpy as np
predict =[]
for i in range(len(result_xgb)) :
    result_xgb_list=result_xgb[i].tolist()
    predict.append(result_xgb_list.index(max(result_xgb_list))
predict

In [ ]:
result = pd.DataFrame(predict,columns=['label'])
result.to_csv('./xgb_2000_tiidf_deep12.csv',index=False)